#  Data prep for the leakage problem

##  Set up the correct version of SQLite3 that allows us to do `row_number()` operations
Remember that you need to do `Menu > Runtime > restart runtime` since the runtime is already loaded

In [ ]:
!curl https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release | tar xz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.7/pysqlite3/_sqlite3.cpython-37m-x86_64-linux-gnu.so \
     /usr/lib/python3.7/lib-dynload/

## Library imports and important settings/constants

In [ ]:
import pandas as pd 
import numpy as np
from types import  SimpleNamespace
import os.path as osp 
from matplotlib import pyplot as plt
from google.colab import drive 
from sqlalchemy import create_engine
import sqlite3 as lite 
print("Installed SQLite version {} at least 3.25".format(lite.sqlite_version))

In [ ]:
YEAR = 2017  #<-- file year
MOUNT_LOCATION = "/content/drive/MyDrive/Docgraph"

#  Namespaces for easy access
PUF = SimpleNamespace(zip=f"{MOUNT_LOCATION}/Medicare_Provider_Util_Payment_PUF_CY{YEAR}.zip", 
                      data=f"Medicare_Provider_Util_Payment_PUF_CY{YEAR}/Medicare_Provider_Util_Payment_PUF_CY{YEAR}.txt", 
                      dir=f"Medicare_Provider_Util_Payment_PUF_CY{YEAR}", 
                      table=f"{MOUNT_LOCATION}/db/PUFF_{YEAR}.csv", 
                      detail=f"{MOUNT_LOCATION}/db/PUFF_DETAIL_{YEAR}.csv")

DOCGRAPH = SimpleNamespace(zip=f"{MOUNT_LOCATION}/DocGraph_Hop_Teaming_{YEAR}_CC.zip", 
                      data=f"DocGraph_Hop_Teaming_{YEAR}_CC/DocGraph_Hop_Teaming_{YEAR}_CC/DocGraph_Hop_Teaming_{YEAR}.csv", 
                      dir=f"DocGraph_Hop_Teaming_{YEAR}_CC", 
                      table=f"{MOUNT_LOCATION}/db/DOCGRAPH_{YEAR}.csv")

_npi_name_lookup = {2014: f"{YEAR}_National_Downloadable_File/National_Downloadable_File.csv", 
                    2015: f"{YEAR}_National_Downloadable_File/Physician_Compare_Databases/National_Downloadable_File.csv", 
                    2016: f"{YEAR}_National_Downloadable_File/National_Downloadable_File.csv", 
                    2017: f"{YEAR}_National_Downloadable_File/Physician_Compare_2014_Group_Practice_Public_Reporting_-_Clinical_Quality_Of_Care.csv"}


 
NPI = SimpleNamespace(zip=f"{MOUNT_LOCATION}/{YEAR}_National_Downloadable_File.zip", 
                      data=_npi_name_lookup[YEAR], 
                      dir=f"{YEAR}_National_Downloadable_File", 
                      table=f"{MOUNT_LOCATION}/db/NPI_{YEAR}.csv") 


#  RVU files are particularly nasty because the government has fluid naming conventions... 
_rvu_name_lookup = {2014:'RVU14A', 2015:'RVU15A', 2016:'RVU16A', 2017:'rvu17a3'}
_rvu_file_lookup = {2014:'PPRRVU14_V1219', 2015: 'PPRRVU15_V1223c', 2016: 'PPRRVU16_V0122', 2017: 'PPRRVU17_V1219'}   
_rvu_zip_name = _rvu_name_lookup[YEAR]
_rvu_file_name = _rvu_file_lookup[YEAR]

RVU = SimpleNamespace(zip=f"{MOUNT_LOCATION}/{_rvu_zip_name}.zip", 
                      data=f"{_rvu_zip_name}/{_rvu_zip_name}/{_rvu_file_name}.csv", 
                      dir=f"{_rvu_zip_name}", 
                      table=f"{MOUNT_LOCATION}/db/RVU_{YEAR}.csv")

#  Mount the drive 
drive.mount('/content/drive')

In [ ]:
#  Postgres concept on AWS
username = ""
host = "amazonaws.com"
#engine = create_engine(f'postgresql://{username}:{password}@{host}:5432/dbmaster')

#  Database assignment... note to self:  make sure not pointing to public Postgres anymore until ready for prime-time
engine = create_engine(f"sqlite:////{MOUNT_LOCATION}/data_{YEAR}.db")

## Build underlying data constructs from flat files located on Google drive

In [ ]:
#  
#  Utility functions 
#
def _get_puf_data(filename: str) -> pd.DataFrame: 
  df = pd.read_csv(filename, 
                  sep='\t', 
                  skiprows=[1], 
                  header=0)
  df.columns = map(str.lower, df.columns)
  df = df.loc[df['nppes_entity_code'] == 'I'].copy() 
  cols = ['npi', 'bene_unique_cnt', 'line_srvc_cnt', 'hcpcs_code']  
  df = df[cols].copy()
  return df 

def _em_ratio(data: pd.DataFrame) -> pd.Series:
  num = np.sum(data.office.values * data.bene_unique_cnt.values)
  denom = np.sum(data.bene_unique_cnt.values)
  return pd.Series(dict(em_ratio=num/denom))

#
#  PUF files -- detail and aggregated for E&M 
#
!unzip {PUF.zip} -d {PUF.dir}
df = _get_puf_data(filename=PUF.data)
df.to_csv(PUF.detail, index=False)
df['office'] = [int(x[:3] == '992') for x in df.hcpcs_code.values]  #Try to capture all the EM codes starting with 992... 
df.drop('hcpcs_code', axis=1, inplace=True)
puf = df.groupby(['npi']).apply(_em_ratio)
puf = puf.reset_index()
puf.to_csv(PUF.table, index=False)



In [ ]:
#
#  NPI files
#
if YEAR != 2017: 
  !unzip {NPI.zip} -d {NPI.dir}
  df = pd.read_csv(NPI.data)
else: 
  df = pd.read_csv("/content/drive/MyDrive/Docgraph/2017-02-Physician_Compare_National_Downloadable_File.csv")
  
df.columns = map(str.lower, df.columns)
columns = ['npi','pac id','last name','first name', 
           'gender','graduation year','primary specialty', 'organization legal name', 'group practice pac id', 
           'number of group practice members','city','state','zip code']
df = df[columns].copy()
df.rename(columns={'number of group practice members': 'group_size', 'zip code': 'zipcode'}, inplace=True)
df.to_csv(NPI.table, index=False)

In [ ]:
#
#  RVU files
#  Note:  Take heed with this... the government files aren't very consistent
#
!unzip {RVU.zip} -d {RVU.dir}
df = pd.read_csv(f"{RVU.data}", header=0, skiprows=[0, 1, 2, 3, 4, 5, 6, 7, 8])
df = df[['HCPCS', 'MOD', 'RVU']].copy()
df['modifier'] = df['MOD'].fillna(0)
df = df.iloc[np.where((df.RVU.values > 0 ) & (df.modifier.values == 0))[0]].copy()
df.drop(['MOD', 'modifier'], axis=1, inplace=True)
df.to_csv(RVU.table, index=False)

In [ ]:

#
#  Docgraph HOPPR file
#
!unzip {DOCGRAPH.zip} -d {DOCGRAPH.dir}

In [ ]:
if YEAR == 2017: 
  docgraph = pd.read_csv("DocGraph_Hop_Teaming_2017_CC/DocGraph_Hop_Teaming_2017_Non_Commercial/DocGraph_Hop_Teaming_2017.csv")
  
else: 
  docgraph = pd.read_csv(DOCGRAPH.data)

In [ ]:
docgraph

In [ ]:
docgraph.drop(['transaction_count', 'std_day_wait'], axis=1, inplace=True)
df_docgraph_ttl = docgraph.groupby('from_npi').agg({'patient_count': 'sum'}).rename(columns={'patient_count':'ttl_patient_count'}).reset_index()
df = docgraph.merge(df_docgraph_ttl, on='from_npi', how='inner')
df.to_csv(DOCGRAPH.table, index=False)

## SQL dataprep

### Create tables from files

#### PUF, NPI and DOCGRAPH files

In [ ]:
puf = pd.read_csv(PUF.table)
npi = pd.read_csv(NPI.table)
# TODO: refactor so that this is done earlier on for consistency
npi.columns = map(lambda s: s.replace(" ", "_"), npi.columns)


In [ ]:
#
#  Create NPI and PUF tables 
#
npi.to_sql("npi", engine, if_exists="replace")
puf.to_sql('puf', engine, if_exists='replace')

In [ ]:
zip_centroid = pd.read_csv(f'{MOUNT_LOCATION}/us-zip-code-latitude-and-longitude.csv', delimiter=";")
zip_centroid.drop(['City', 'State', 'Timezone', 'Daylight savings time flag', 'geopoint'], axis=1, inplace=True)
zip_centroid.to_sql("zipcode", engine, if_exists="replace")

In [ ]:
docgraph = pd.read_csv(DOCGRAPH.table)
docgraph.drop(['ttl_patient_count'], axis=1, inplace=True)  # Don't think we ever use this 

In [ ]:
#
#  Create a docgraph table comprised of the network TO radiologist community... this is huge so takes a long time, change codes as needed: https://www.cms.gov/Medicare/Quality-Initiatives-Patient-Assessment-Instruments/Care-Compare-DAC-Initiative/Frequently-Asked-Questions
#
query = """
select distinct npi 
from npi 
where primary_specialty in 
(
  'INTERVENTIONAL RADIOLOGY', 'DIAGNOSTIC RADIOLOGY', 'NUCLEAR MEDICINE' 
)
"""
docs_we_care_about = pd.read_sql_query(query, engine)
_docgraph = docgraph.merge(docs_we_care_about, how='inner', left_on="to_npi", right_on="npi")
_docgraph.to_sql("docgraph", engine, if_exists="replace")

In [ ]:
_docgraph

#### Radiology academic centers 

In [ ]:
#
#  Academic centers from Harvey  
#  
df = pd.read_csv(f'{MOUNT_LOCATION}/Radiology_AcademicPractices.csv')
df.columns = map(lambda s: s.replace(" ", "_").lower().replace("?", "_ind"), df.columns)
df = df.loc[df['academic_ind'] == 1].copy().reset_index()
#df.drop(['organization_name', 'city', 'state', 'academic_ind'])
#df.to_sql("academic", engine, if_exists="replace")

In [ ]:
df.drop(['index', 'organization_name', 'city', 'state', 'academic_ind'], axis=1, inplace=True)

In [ ]:
df.to_sql("academic", engine, if_exists="replace")

#### SDI 

In [ ]:
#
#  SDI features
#
df = pd.read_csv(f'{MOUNT_LOCATION}/ACS2015_zctaallvars.csv')
df['zipcode'] = [str(s).rjust(5, '0') for s in df.zcta.values]
df = df[['zipcode','population','sdi_score','highneeds_score']].copy()
df.to_sql("sdi", engine, if_exists="replace")

#### RVU

In [ ]:
#  
#  RVU table 
#
rvu = pd.read_csv(RVU.table)
rvu.to_sql("rvu", engine, if_exists="replace")

#### PUF detail

In [ ]:
#
#  PUF detail table 
#
puf_detail = pd.read_csv(PUF.detail)
puf_detail.to_sql("puf_detail", engine, if_exists="replace")

### Urban_zip file 

In [ ]:
df = pd.read_csv(f'{MOUNT_LOCATION}/rural_urban.txt', delimiter='\t')
df['zcta'] = [str(s).rjust(5, '0') for s in df.ZCTA5.values]
df['urban'] = df['urban_rural-percent-urban_population-of-total_population']
urban = df[['zcta', 'urban']].copy()

df = pd.read_csv(f'{MOUNT_LOCATION}/Zip_to_zcta_crosswalk_2020.csv')
df['zcta'] = [str(s).rjust(5, '0') for s in df.ZCTA.values]
df['zip'] = [str(s).rjust(5, '0') for s in df.ZIP_CODE.values]
zcta_map = {a: b for a, b in zip(df.zcta.values, df.zip.values)}

urban['zip'] = [zcta_map.get(zcta, '99999') for zcta in urban.zcta.values]

In [ ]:
urban.to_sql("zip_urban", engine, if_exists="replace")

### Derived tables

#### Create the `valid_referer` table, those that have `em_ratio > 0`

In [ ]:
#
#  Isolate only valid referring providers... 
#
engine.execute("drop table if exists valid_referer")
query = """
create table valid_referer as 
select distinct npi 
from puf 
where em_ratio > 0 
"""
engine.execute(query)

#### Get only a single pairing of npi to group practice to create `npi_x_group`
Here we make the choice to limit it to a single group... 

In [ ]:
#
#   Get only a single pairing of npi to group practice... 
#  
engine.execute("drop table if exists npi_x_group")
query = """
create table npi_x_group as 
select npi, group_practice_pac_id
from 
(
  select npi, group_practice_pac_id, group_size, 
    row_number() over (partition by npi order by group_size desc) as rn
  from 
  (
    select distinct npi, group_practice_pac_id, group_size
    from npi 
  ) x
) y 
where y.rn = 1 
"""
engine.execute(query)

#### npi_x_group_weight

In [ ]:
engine.execute("drop table if exists npi_x_group_weight")
query = """
create table npi_x_group_weight as 
select z.*
from 
(
select x.npi, x.group_practice_pac_id,
    sum(case when trim(pc.hcpcs_code) in ('77084', '75554', '75555', '75552', '75553', '75556', '75565', '75557', '75558', '75560', '75559', '75564', '75562', '75563', '75561', '76375', '74177', '74176', '74178', '74160', '74170', '74150', '74174', '74175', '71275', '70496', '73706', '70498', '72191', '73206', '76071', '76070', '77079', '77078', '72126', '72127', '72125', '74263', '74262', '74261', '76360', '76370', '76355', '76362', '77013', '77012', '77014', '77011', '70460', '70470', '70450', '75572', '75571', '75573', '76380', '73701', '73702', '73700', '72132', '72133', '72131', '70487', '70488', '70486', '70481', '70482', '70480', '72193', '72194', '72192', '70491', '70492', '70490', '72129', '72130', '72128', '71260', '71270', '71250', '73201', '73202', '73200', '75635', '75574', '77022', '77021', '74185', '71555', '70545', '70546', '70544', '73725', '70548', '70549', '70547', '72198', '72159', '73225', '76393', '76394', '74182', '74183', '74181', '73722', '73723', '73721', '73222', '73223', '73221', '76400', '70552', '70551', '70553', '70554', '70555', '70558', '70559', '70557', '77059', '77058', '76094', '76093', '71551', '71552', '71550', '73719', '73720', '73718', '70542', '70543', '70540', '72196', '72197', '72195', '76390', '72142', '72156', '72141', '72149', '72158', '72148', '72147', '72157', '72146', '73220', '73219', '73218', '76497', '76498') 
      then pc.bene_unique_cnt else 0 end) as rads_bene, 
    sum(pc.bene_unique_cnt) as ttl_bene
  from npi_x_group x 
    inner join puf_detail pc 
    on x.npi = pc.npi
  group by x.npi, x.group_practice_pac_id
) z
where rads_bene > 0
"""
engine.execute(query)

#### Rad group weighting


In [ ]:
engine.execute("drop table if exists group_weight")
query = """
create table group_weight as 
select group_practice_pac_id, rads_bene, ttl_bene, 
  ((rads_bene*1.0)/ttl_bene) as group_weight
from 
(
  select group_practice_pac_id, 
    sum(rads_bene) as rads_bene, 
    sum(ttl_bene) as ttl_bene
  from npi_x_group_weight
  group by group_practice_pac_id
) x 
"""
engine.execute(query)

#### Associate the `from_npi` (referrer) to a group, and sum up the number of patients referred

In [ ]:
#
#  Intermediate step... associate the referral to a group and calculate some statistics 
#
engine.execute("drop table if exists from_npi_x_to_group")
query = """
create table from_npi_x_to_group as 
select d.from_npi, n.group_practice_pac_id, 
  sum(d.patient_count) as patient_count, 
  avg(d.average_day_wait) as average_day_wait
from docgraph d
  inner join npi_x_group n 
  on d.to_npi = n.npi 
  inner join valid_referer v 
  on d.from_npi = v.npi
where n.group_practice_pac_id is not null 
  and n.group_practice_pac_id <> ''
group by d.from_npi, n.group_practice_pac_id
"""
engine.execute(query)

#### Associate average wait time to to_NPI

In [ ]:
#
#  Intermediate step... associate the referral to a group and calculate some statistics 
#
engine.execute("drop table if exists group_wait_time")
query = """
create table group_wait_time as 
select n.group_practice_pac_id, 
  avg(d.average_day_wait) as average_day_wait
from docgraph d
  inner join npi_x_group n 
  on d.to_npi = n.npi 
  inner join valid_referer v 
  on d.from_npi = v.npi
where n.group_practice_pac_id is not null 
  and n.group_practice_pac_id <> ''
group by n.group_practice_pac_id
"""
engine.execute(query)

#### `from_npi_ttl` Total up the patient counts for each `from_npi`

In [ ]:
#
#  Total up the patient counts for referrers... 
#
engine.execute("drop table if exists from_npi_ttl")
query = """
create table from_npi_ttl as 
select from_npi, sum(patient_count) as ttl_patient_count
from from_npi_x_to_group 
group by from_npi 
"""
engine.execute(query)

#### `from_npi_x_to_group_ttl` put the activity summries together

In [ ]:
#
#  Get the referral activity for each from_npi 
#
engine.execute("drop table if exists from_npi_x_to_group_ttl")
query = """
create table from_npi_x_to_group_ttl as 
select x.from_npi, x.group_practice_pac_id, x.patient_count, y.ttl_patient_count, x.average_day_wait
from from_npi_x_to_group x 
  inner join from_npi_ttl y 
  on x.from_npi = y.from_npi
"""
engine.execute(query)

#### `to_group_feature` some features about the groups 

In [ ]:
engine.execute("drop table if exists to_npi_temp")
query = """
create table to_npi_temp as 
select distinct to_npi as npi 
from docgraph
"""
engine.execute(query)

In [ ]:
engine.execute("drop table if exists to_npi_detail")
query = f"""
create table to_npi_detail as 
select n.npi, n.group_practice_pac_id, 
  max(case when gender = 'M' then 1 else 0 end) as ind_male, 
  avg({YEAR} - graduation_year) as yrs_graduation
from to_npi_temp x 
  inner join npi n 
  on x.npi = n.npi
group by n.npi, n.group_practice_pac_id 	
"""
engine.execute(query)

In [ ]:
engine.execute("drop table if exists to_group_feature")
query = f"""
create table to_group_feature as 
select group_practice_pac_id, count(distinct npi) as n_radiologists, avg(ind_male) as avg_male, avg(yrs_graduation) as avg_yrs_graduation
from to_npi_detail
group by group_practice_pac_id
"""
engine.execute(query)

####  `referral_feature` for features about each referrer (`from_npi`)

In [ ]:
#
#  Features for the referring NPI... making sure only 1 per npi
#
engine.execute("drop table if exists referral_feature")
query = f"""
create table referral_feature as  
select npi, zipcode, ind_male, group_size, primary_specialty, yrs_graduation
from 
(
select n.*, 
    row_number() over (partition by n.npi order by group_size desc, zipcode) as rn 
  from 
  (
    select npi, 
      substr('00000' || substr(cast(zipcode as text), 1, 5), -5, 5) as zipcode, 
      case when gender = 'M' then 1 else 0 end as ind_male, 
      group_size, 
      primary_specialty, 
      ({YEAR} - graduation_year) as yrs_graduation
    from npi
  ) as n 
) n1 
where n1.rn = 1
"""
engine.execute(query)

####  `npi_rvu` estimate the RVU for each provider 

In [ ]:
#
#  Create an estimate of RVU for each npi 
#
engine.execute("drop table if exists npi_rvu")
query = """
create table npi_rvu as 
select p.npi, sum(coalesce(r.rvu, 0.0) * p.line_srvc_cnt) as rvu 
from puf_detail p 
  left join rvu r 
  on p.hcpcs_code = r.hcpcs 
group by p.npi 
"""
engine.execute(query)

#### `group_rvu` to estimate the RVU for each of our groups

In [ ]:
#
#  Estimate RVU for our to_groups
# 
engine.execute("drop table if exists group_rvu")
query = """
create table group_rvu as 
select x.group_practice_pac_id, sum(rvu) as rvu 
from npi_x_group x 
  inner join npi_rvu r 
  on x.npi = r.npi 
group by x.group_practice_pac_id
"""
engine.execute(query)

#### `group_zip` to associate a single zip to a group. 

In [ ]:
#
#  Group zip centroid
#
engine.execute("drop table if exists group_zip")
query = """
create table group_zip as 
select group_practice_pac_id, zipcode 
from 
(
  select a.*, 
    row_number() over (partition by group_practice_pac_id order by group_size desc, zipcode) as rn 
  from 
  (
    select distinct group_practice_pac_id, group_size, 
      substr('00000' || substr(cast(zipcode as text), 1, 5), -5, 5) as zipcode
    from to_npi_temp x 
      inner join npi n 
      on x.npi = n.npi
  ) a 
) b 
where b.rn = 1 
"""
engine.execute(query)

#### `group_leakage` to associate leakage to a group 

In [ ]:
engine.execute("drop table if exists group_leakage")
query = """
create table group_leakage as 
select
  x.group_practice_pac_id, 
  sum(x.patient_count*1.0000) as n_referral, 
  sum(x.ttl_patient_count*1.0000) as n_potential_referral,  
  sum(x.patient_count*1.0000)/sum(x.ttl_patient_count*1.0000) as leakage
from from_npi_x_to_group_ttl x 
where x.group_practice_pac_id is not null 
and length(x.group_practice_pac_id) > 3 
and x.ttl_patient_count > 0
and x.patient_count > 0 
group by x.group_practice_pac_id
"""
engine.execute(query)

#  Feature space 

In [ ]:
query = """
select
  (x.patient_count*1.0000)/(x.ttl_patient_count*1.0000) as leakage, 
  x.from_npi, 
  nxg.group_practice_pac_id as from_group_practice_pac_id, 
  x.group_practice_pac_id as to_group_practice_pac_id, 
  rf.zipcode,
  x.patient_count, x.ttl_patient_count,
  rf.ind_male, rf.group_size, rf.primary_specialty, rf.yrs_graduation, 
  f.n_radiologists, x.average_day_wait, 
  coalesce(z.latitude, 0) as latitude, 
  coalesce(z.longitude, 0) as longitude, 
  coalesce(g.rvu, 0)/f.n_radiologists as normalized_rvu, 
  coalesce(zz.latitude, 0) as to_latitude, 
  coalesce(zz.longitude, 0) as to_longitude, 
  s.population,s.sdi_score,s.highneeds_score, 
  case when ac.group_id is not null then 1 else 0 end as ind_academic, 
  case when nxg.group_practice_pac_id = x.group_practice_pac_id then 1 else 0 end as ind_internal_referral
from from_npi_x_to_group_ttl x 
  inner join to_group_feature f 
  on x.group_practice_pac_id = f.group_practice_pac_id
  inner join referral_feature rf 
  on x.from_npi = rf.npi 
  left join zipcode z 
  on rf.zipcode = z.zip 
  left join group_rvu g
  on x.group_practice_pac_id = g.group_practice_pac_id
  inner join group_zip gz 
  on x.group_practice_pac_id = gz.group_practice_pac_id
  left join zipcode zz
  on gz.zipcode = zz.zip 
  inner join sdi s 
  on rf.zipcode = s.zipcode
  inner join npi_x_group nxg 
  on x.from_npi = nxg.npi
  left join 
  (
    select distinct group_id 
    from academic 
  ) ac on x.group_practice_pac_id = ac.group_id
where f.n_radiologists > 1 
and x.group_practice_pac_id is not null 
and length(x.group_practice_pac_id) > 3 
and x.ttl_patient_count > 0
and g.rvu > 0 
and x.patient_count > 0 
"""
df = pd.read_sql_query(query, engine)

## Geo distance

In [ ]:
from geopy import distance
d = df[['latitude', 'longitude', 'to_latitude', 'to_longitude']].copy()
d['distance'] = d.apply(lambda x: distance.distance((x['latitude'], x['longitude']), (x['to_latitude'], x['to_longitude'])).km, axis=1)

In [ ]:
df['distance'] = d.distance.values

In [ ]:
df.dropna(axis=0, inplace=True)

In [ ]:
df.to_csv(f'{MOUNT_LOCATION}/db/X_{YEAR}.csv', index=False)

In [ ]:
df

## Prepare it at the radiologist group level

In [ ]:
query = """
select
  x.group_practice_pac_id, x.n_referral, x.n_potential_referral, x.leakage, 
  f.n_radiologists, 
   coalesce(g.rvu, 0)/f.n_radiologists as normalized_rvu, 
  coalesce(zz.latitude, 0) as to_latitude, 
  coalesce(zz.longitude, 0) as to_longitude, 
  case when ac.group_id is not null then 1 else 0 end as ind_academic, 
  gwt.average_day_wait, 
  coalesce(zu.urban, 0) as pct_urban,

  coalesce(s.population, 0) as population,
  coalesce(s.sdi_score,0) as sdi_score,
  coalesce(s.highneeds_score, 0) as highneeds_score, 

  gw.rads_bene, gw.ttl_bene, 
  gw.rads_bene, ttl_bene, 
  ((rads_bene*1.0)/ttl_bene) as group_weight

from group_leakage x
  inner join to_group_feature f 
  on x.group_practice_pac_id = f.group_practice_pac_id
  left join group_rvu g
  on x.group_practice_pac_id = g.group_practice_pac_id
  inner join group_zip gz 
  on x.group_practice_pac_id = gz.group_practice_pac_id
  left join zipcode zz
  on gz.zipcode = zz.zip 
  left join 
  (
    select distinct group_id 
    from academic 
  ) ac on x.group_practice_pac_id = ac.group_id
  inner join group_wait_time gwt
  on x.group_practice_pac_id = gwt.group_practice_pac_id 
  inner join zip_urban zu 
  on gz.zipcode = zu.zip

  inner join sdi s 
  on gz.zipcode = s.zipcode

  inner join group_weight gw
  on x.group_practice_pac_id = gw.group_practice_pac_id


where gwt.average_day_wait > 0 
"""
df = pd.read_sql_query(query, engine)

In [ ]:
df

In [ ]:
np.average(1-df.leakage.values, weights=df.group_weight.values), np.average(1-df.leakage.values), np.mean((1.-df.leakage.values)*df.group_weight.values)

In [ ]:
df['weighted_leakage'] = (1. - df.leakage.values) * df.group_weight.values

In [ ]:
df

In [ ]:
plt.hist(df.weighted_leakage.values), df.weighted_leakage.values.mean()

In [ ]:
df.dropna(axis=0, inplace=True)
df.to_csv(f'{MOUNT_LOCATION}/db/X_group_{YEAR}_weight.csv', index=False)

In [ ]:
plt.hist(df.leakage, bins=100, density=True);

In [ ]:
import seaborn as sns

sns.kdeplot(df.leakage, df.n_radiologists)

In [ ]:
df

## EDA

# Analysis of top CPT codes

In [ ]:
query = """
select * from docgraph
"""
#   where trim(pc.hcpcs_code) in ('77084', '75554', '75555', '75552', '75553', '75556', '75565', '75557', '75558', '75560', '75559', '75564', '75562', '75563', '75561', '76375', '74177', '74176', '74178', '74160', '74170', '74150', '74174', '74175', '71275', '70496', '73706', '70498', '72191', '73206', '76071', '76070', '77079', '77078', '72126', '72127', '72125', '74263', '74262', '74261', '76360', '76370', '76355', '76362', '77013', '77012', '77014', '77011', '70460', '70470', '70450', '75572', '75571', '75573', '76380', '73701', '73702', '73700', '72132', '72133', '72131', '70487', '70488', '70486', '70481', '70482', '70480', '72193', '72194', '72192', '70491', '70492', '70490', '72129', '72130', '72128', '71260', '71270', '71250', '73201', '73202', '73200', '75635', '75574', '77022', '77021', '74185', '71555', '70545', '70546', '70544', '73725', '70548', '70549', '70547', '72198', '72159', '73225', '76393', '76394', '74182', '74183', '74181', '73722', '73723', '73721', '73222', '73223', '73221', '76400', '70552', '70551', '70553', '70554', '70555', '70558', '70559', '70557', '77059', '77058', '76094', '76093', '71551', '71552', '71550', '73719', '73720', '73718', '70542', '70543', '70540', '72196', '72197', '72195', '76390', '72142', '72156', '72141', '72149', '72158', '72148', '72147', '72157', '72146', '73220', '73219', '73218', '76497', '76498') 
#select pc.hcpcs_code, sum(pc.bene_unique_cnt) as sum_bene_unique_cnt
#  from docgraph x 
#    inner join puf_detail pc 
#    on x.to_npi = pc.npi
#  group by pc.hcpcs_code]

#select pc.hcpcs_code, sum(pc.bene_unique_cnt) as sum_bene_unique_cnt
#  from puf_detail pc
#    inner join (select distinct to_npi from docgraph) x 
#    on x.to_npi = pc.npi
#  group by pc.hcpcs_code

df = pd.read_sql_query(query, engine)

In [ ]:
query = """SELECT name FROM sqlite_master WHERE type='table';
"""

df = pd.read_sql_query(query, engine)
df


In [ ]:
query = """SELECT * FROM docgraph
"""

df = pd.read_sql_query(query, engine)
df




In [ ]:
df.to_csv(f'{MOUNT_LOCATION}/db/group_zip_{YEAR}.csv', index=False)

In [ ]:
df.sort_values('sum_bene_unique_cnt', ascending=False, inplace=True)

In [ ]:
df = df.iloc[:50].copy()
df

In [ ]:
from google.colab import files
df.to_csv ("docgraph2017_rad_dataframe.csv", index = False, header=True)




In [ ]:
drive.mount('/content/drive')
path = '/content/drive/My Drive/docgraph2017_rad_dataframe.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  df.to_csv(f)

In [ ]:
X = df[['leakage', 'n_radiologists', 'average_day_wait', 'normalized_rvu', 'distance', 'ind_academic']].copy()

In [ ]:
X

In [ ]:
X = X.fillna(0)

In [ ]:
X

In [ ]:
import seaborn as sns

In [ ]:
sns.set_theme(style="ticks")

sns.pairplot(X, hue="ind_academic")